In [99]:
# Set Up the Flask Weather App
##  Import our dependencies
import datetime as dt
import numpy as np
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from flask import Flask, jsonify


In [100]:
# Set Up the Database
engine = create_engine("sqlite:///hawaii.sqlite")

In [101]:
## Reflect the database into our classes:
Base = automap_base()
Base.prepare(engine, reflect=True)

In [102]:
## Create a variable for each of the classes so that we can reference them later:
Measurement = Base.classes.measurement
Station = Base.classes.station

In [103]:
## Create a session link from Python to our database:
session = Session(engine)

In [104]:
## Set Up Flask
app = Flask(__name__)

In [105]:
# Create the Welcome Route
## Define the welcome route:
@app.route("/")
## Add the routing information for each of the other routes:
def welcome():
    return(
    '''
    Welcome to the Climate Analysis API!
    Available Routes:
    /api/v1.0/precipitation
    /api/v1.0/stations
    /api/v1.0/tobs
    /api/v1.0/temp/start/end
    ''')
### Run the code:
##if __name__ == "__main__":
##    app.run()

In [106]:
## Precipitation Route
@app.route("/api/v1.0/precipitation")
def precipitation():
   prev_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)
   precipitation = session.query(Measurement.date, Measurement.prcp).\
    filter(Measurement.date >= prev_year).all()
   precip = {date: prcp for date, prcp in precipitation}
   return jsonify(precip)
### Run the code:
##if __name__ == "__main__":
##    app.run()

In [107]:
## Stations Route:
@app.route("/api/v1.0/stations")
def stations():
    results = session.query(Station.station).all()
    stations = list(np.ravel(results))
    return jsonify(stations=stations)
##if __name__ == "__main__":
##    app.run()

In [108]:
## Monthly Temperature Route:
@app.route("/api/v1.0/tobs")
def temp_monthly():
    prev_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)
    results = session.query(Measurement.tobs).\
      filter(Measurement.station == 'USC00519281').\
      filter(Measurement.date >= prev_year).all()
    temps = list(np.ravel(results))
    return jsonify(temps=temps)
### Run the code:
##if __name__ == "__main__":
##    app.run()

In [109]:
## Statistics Route
@app.route("/api/v1.0/temp/<start>")
@app.route("/api/v1.0/temp/<start>/<end>")
def stats(start=None, end=None):
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]

    if not end:
        results = session.query(*sel).\
            filter(Measurement.date >= start).all()
        temps = list(np.ravel(results))
        return jsonify(temps)

    results = session.query(*sel).\
        filter(Measurement.date >= start).\
        filter(Measurement.date <= end).all()
    temps = list(np.ravel(results))
    return jsonify(temps)
### Run the code:
if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Feb/2022 17:16:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Feb/2022 17:16:26] "GET /api/v1.0/temp/2017-06-01/2017-06-30 HTTP/1.1" 200 -
